# Generate Parameter Screen

Author: Jacob Parres-Gold (jacobparresgold@gmail.com)

Last Revised: 20231019

This notebook performs a parameter screen of networks, considering 1 or 2 monomers as the input.

In [1]:
import itertools
import math
import numpy as np
import os
import pathlib
import scipy.stats
import sys

# EQTK to calculate equilibrium concentrations - install instructions here https://eqtk.github.io/getting_started/eqtk_installation.html
import eqtk

# Ray for parallelization - install instructions here https://docs.ray.io/en/latest/ray-overview/installation.html
import ray

# Networkx for graphs - install instructions here https://networkx.org/documentation/stable/install.html
import networkx as nx

In [2]:
# Load utility functions from dimer_network_utilities.py
utilities_path = str(pathlib.Path('C:\\Users','jacob','Local_Coding',\
                                  'ElowitzLab','Promiscuous_Networks','Notebooks','Paper_Notebooks')) # Directory of your utilities.py file
sys.path.append(utilities_path)
from dimer_network_utilities import *

In [3]:
# Initialize ray
_ = ray.init(ignore_reinit_error=True)

In [ ]:
# To shut down ray:
ray.shutdown()

In [ ]:
# Print package versions
import pkg_resources
imports = list(set(get_imports()))
requirements = []
for m in pkg_resources.working_set:
    if m.project_name in imports and m.project_name!="pip":
        requirements.append((m.project_name, m.version))

print("Package versions:")
print("python={}".format(sys.version.split('|')[0].replace(' ','')))
for r in requirements:
    print("{}={}".format(*r))

Package versions:
python=3.8.13
eqtk=0.1.2
matplotlib=3.5.1
networkx=2.7.1
numpy=1.21.5
pandas=1.4.1
scipy=1.10.1


## Functions for Simulations

In [4]:
def run_eqtk(N, C0, params, acc_monomer_ind):
    """
    Run eqtk.solve given the input stoichiometry matrix, initial concentrations, and parameters.

    See eqtk.solve documentation for more details on the syntax.
    (https://eqtk.github.io/user_guide/generated/highlevel/eqtk.solve.html)

    Parameters
    ----------
    N : Array-like, shape (num_rxns, num_species)
        Input stoichiometry matrix
    C0 : Array-like, shape (num_simulation_points, num_species)
        Initial concentrations of all species for each simulation point.
        Accessory monomer levels will be set based on params.
    params : List-like, len num_combos_with_replacement(m,2) + (m-1)
        Parameters including Kij values and accessory monomer levels
    acc_monomer_ind : int  
        Index of accessory monomers in species list
    
    Returns
    -------
    C: Array-like
        Equilibrium concentrations of all species
    """
    num_rxns = N.shape[0]
    # Extract Kij values
    K = params[:num_rxns]
    # Set accessory monomer levels
    C0[:,acc_monomer_ind] = params[num_rxns:]

    return eqtk.solve(c0=C0, N=N, K=K)

In [5]:
@ray.remote
def simulate_networks(m, num_inputs, param_sets, t = 12, input_lb = -3, input_ub = 3):
    """
    Run simulations for dimer networks, over a titration of concentrations for each input monomer. 
    When >1 input monomers are varied (e.g., a 2D matrix), the first monomer is sampled outermost, 
    while the second is sampled inner to that.
    
    Parameters
    ----------
    m : int. 
        Number of monomer species in the network.
    num_inputs: int
        Number of monomers to count as inputs (those titrated). 
        E.g., if num_inputs=2, the first two monomers will be treated as inputs.
    t : int. Default 12. 
        Number of values to titrate each input monomer species. 
        Values spaced evenly on a log10 scale
    input_lb : int. Default -3
        lower bound for titrating the input monomer species. Log10 scale
    input_ub : int. Default 3
        upper bound for titrating the input monomer species.Llog10 scale
    param_sets : Array-like, shape (num_sets, num_parameters)
        Parameter sets for simulating multiple dimerization networks.  
    Returns
    -------
    C0 : Array-like, shape (t, number of species)
        Initial concentration array used for eqtk.solve
    S_all : Array-like, shape (t, number of species, num_sets) 
        Equlibrium concentration of all species
        for all parameter sets and each input titration point.
        
    """
    # Create stoichiometry matrix 
    num_sets = param_sets.shape[0]
    N = make_nXn_stoich_matrix(m)
    num_rxns = N.shape[0]

    # Create initial concentration array
    M0_min = [input_lb]*num_inputs + [0] * (m-num_inputs) # Species concentrations at min of inputs
    M0_max = [input_ub]*num_inputs + [0] * (m-num_inputs) # Species concentrations at max of inputs
    num_conc = [t]*num_inputs + [1] * (m-num_inputs) # Number of concentrations to titrate for each species
    C0 = make_C0_grid(m, M0_min=M0_min, M0_max=M0_max, num_conc=num_conc) 
        
    acc_monomer_ind = np.arange(num_inputs,m) # Indices of accessory monomers
    S_all = np.zeros((C0.shape[0], C0.shape[1], num_sets))
    # For each parameter set, run eqtk.solve
    for pset_index, pset in enumerate(param_sets):
        S_all[:,:,pset_index] = run_eqtk(N, C0.copy(), pset, acc_monomer_ind)
    return C0,S_all  
                

## Functions for 1D Screen

In [6]:
def sample_param_sets(m,num_sets,K_log_range,A_log_range,null_K=1e-10,null_K_homo_fraction=0.25,seed=42):
    '''
    Sample parameter sets on a continuous log-uniform range (with Latin Hypercube), making sure to sample evenly across networks of different connectivities
    
    Parameters
    ----------
    m : int
        Number of monomers in the network
    num_sets : int
        Number of parameter sets to sample
    K_log_range : list-like, len 2
        Lower and upper bounds for log10 of Kij values
    A_log_range : list-like, len 2
        Lower and upper bounds for log10 of accessory monomer levels
    null_K : float, default 1e-10
        Value to use for Kij values when no dimerization occurs
        (this is done for convenience; it is easier to set K to some small value
        than to erase the reaction when running EQTK)
    null_K_homo_fraction : float, default 0.25
        The fraction of homodimerization reactions that will be set to null_K (i.e., not occur)
    seed : int, default 42

    Returns
    -------
    param_sets : Array-like, shape (num_sets, num_combos_with_replacement(m,2) + (m-1))
    '''
    rng = np.random.default_rng(seed=seed)

    # Calculate number of parameter sets to create, such
    # that different connectivities are sampled evenly
    num_possible_num_edges = num_combos(m,r=2) - (m-1) + 1
    num_sets_per_num_edges = math.ceil(num_sets/num_possible_num_edges)
    num_sets = int(num_sets_per_num_edges*num_possible_num_edges)

    # Create keys for which dimers are homodimers vs. heterodimers
    edge_key = {}
    homo_edges = []
    hetero_edges = []

    for i,combo in enumerate(itertools.combinations_with_replacement(range(m),r=2)):
        edge_key[(combo[0],combo[1])] = i
        if combo[0]==combo[1]:
            homo_edges.append(i)
        else:
            hetero_edges.append(i)

    param_sets = np.concatenate((np.full((num_sets,num_combos_with_replacement(m,r=2)),null_K),np.zeros((num_sets,m-1))),axis=1)
    max_num_edges_for_disconnected = math.comb(m-1,2)

    for num_edges_i,num_edges in enumerate(range(m-1,num_combos(m,r=2)+1)):
        # For different connectivities (number of edges), do the following:
        for param_set in range(num_sets_per_num_edges):
            # For each parameter set, do the following:
            graph = nx.gnm_random_graph(n=m,m=num_edges) # Generate a Networkx graph
            if num_edges<=max_num_edges_for_disconnected:
                # If false, graph is always connected
                while not nx.is_connected(graph):
                    # Regenerate graph until connected
                    graph = nx.gnm_random_graph(n=m,m=num_edges)
            edge_indices = [edge_key[x] for x in graph.edges]
            param_sets[(num_edges_i*num_sets_per_num_edges)+param_set,edge_indices] = 1
        # Fill all heterodimer K's with hypercube sampling
        lhs_sampler =  scipy.stats.qmc.LatinHypercube(d=num_edges, centered=False, seed=seed)
        hetero_Ks = lhs_sampler.random(n=num_sets_per_num_edges)
        hetero_Ks = 10**scipy.stats.qmc.scale(hetero_Ks, K_log_range[0], K_log_range[1])
        where_1 = np.where(param_sets[(num_edges_i*num_sets_per_num_edges):((num_edges_i+1)*num_sets_per_num_edges),:]==1)
        param_sets[(where_1[0]+(num_edges_i*num_sets_per_num_edges),where_1[1])] = hetero_Ks.flatten()

    # Generate homodimer K's with hypercube sampling
    lhs_sampler =  scipy.stats.qmc.LatinHypercube(d=len(homo_edges), centered=False, seed=seed)
    homo_Ks = lhs_sampler.random(n=num_sets)
    param_sets[:,homo_edges] = 10**scipy.stats.qmc.scale(homo_Ks, K_log_range[0], K_log_range[1])
    
    # Fill some homodimer K's with null K
    null_K_homo_indices = np.array(list(itertools.product(range(num_sets),homo_edges)))[rng.choice(num_sets*len(homo_edges),size=(math.ceil(null_K_homo_fraction*num_sets*len(homo_edges)),),replace=False)]
    null_K_homo_indices = (null_K_homo_indices[:,0],null_K_homo_indices[:,1])
    param_sets[null_K_homo_indices] = null_K
    
    # Generate A's with hypercube sampling
    lhs_sampler =  scipy.stats.qmc.LatinHypercube(d=m-1, centered=False, seed=seed)
    As = lhs_sampler.random(n=num_sets)
    param_sets[:,-(m-1):] = 10**scipy.stats.qmc.scale(As, A_log_range[0], A_log_range[1])
    
    return param_sets

In [ ]:
def dimers_from_param_set(param_set,edges_to_count,min_affinity=1e-5):
    # Return a boolean matrix identifying whether each dimer was "possible" (edge present) in each network
    return np.array([param_set[edge]>=min_affinity for edge in edges_to_count])

## Perform 1D Screen

In [ ]:
# Define directory to save to
out_dir = str(pathlib.Path('C:\\Users','jacob','Downloads'))

if not os.path.isdir(out_dir):
    os.mkdir(out_dir)

In [ ]:
num_inputs = 1
K_log_range = [-5,7] # Lower and upper bounds for log10 of Kij values
A_log_range = [-3,3] # Lower and upper bounds for log10 of accessory monomer levels
null_K=1e-10 # Value to use for Kij values when no dimerization occurs
                # (this is done for convenience; it is easier to set K to some small value
                # than to erase the reaction when running EQTK)
null_K_homo_fraction=0.25 # The fraction of homodimerization reactions that will be set to null_K (i.e., not occur)
num_sets=int(1e6) # Approximate number of parameter sets to simulate 
                    # (more may be simulated to ensure equal numbers of parameter sets for all network connectivities)
input_lb = -3 # Lower bound for log10 of input concentrations
input_ub = 3 # Upper bound for log10 of input concentrations
min_dynamic_range = 10 # Minimum dynamic range of the output responses (all other responses will be filtered out)
round_low_concs_to = 10**-3 # Concentrations below this value will be rounded to this value
t = 30 # Number of input concentrations to simulate
max_curves_per_chunk_to_simulate=5000 # Maximum number of curves to simulate in a single "chunk"
max_networks_per_S_all=200000 # Maximum number of networks to save in a single S_all file

In [ ]:
m_list = list(range(2,13)) # Network sizes to simulate

In [ ]:
S_all_filenames_by_m = []
all_curves_per_Sall_chunk_by_m = []
unfiltered_curves_per_Sall_chunk_by_m = []
nonzero_curves_per_Sall_chunk_by_m = []
filtered_curves_per_Sall_chunk_by_m = []

# Iterate over network sizes
for m in m_list:
    specific_out_dir = str(pathlib.Path(out_dir,f'{m}M'))
    if not os.path.isdir(specific_out_dir):
        os.mkdir(specific_out_dir)
    rng = np.random.default_rng(seed=42)
    # Sample Param Sets
    print(f"Sampling parameter sets for m={m}")
    param_sets = sample_param_sets(m=m,\
                               num_sets=num_sets,\
                               K_log_range=K_log_range,\
                               A_log_range=A_log_range,\
                               null_K=null_K,\
                               null_K_homo_fraction=null_K_homo_fraction,\
                               seed=42)
    dimers_by_param_set = np.apply_along_axis(dimers_from_param_set,\
                                                  axis=1,\
                                                  arr=param_sets,\
                                                  edges_to_count=list(range(num_combos_with_replacement(m,2))))
    num_sets = param_sets.shape[0]
    np.save(str(pathlib.Path(specific_out_dir,f'K_log_range.npy')),np.array(K_log_range))
    np.save(str(pathlib.Path(specific_out_dir,f'A_log_range.npy')),np.array(A_log_range))
    np.save(str(pathlib.Path(specific_out_dir,f'K_A_param_sets.npy')), param_sets)
    np.save(str(pathlib.Path(specific_out_dir,f'dimers_by_param_set.npy')),dimers_by_param_set)
    # Chunk & simulate in parallel
    # There are two levels of chunking here:
    # 1. Each S_all matrix will only consider ~200,000 networks, for memory limitations
    # 2. We will further chunk that up for simulation to allow parallelization
    num_S_all_chunks = math.ceil(param_sets.shape[0]/max_networks_per_S_all)
    S_all_processed_filtered_files = []
    all_curves_per_Sall_chunk = []
    unfiltered_curves_per_Sall_chunk = []
    nonzero_curves_per_Sall_chunk = []
    filtered_curves_per_Sall_chunk = []
    for S_all_chunk in range(num_S_all_chunks):
        if S_all_chunk==num_S_all_chunks:
            param_sets_S_all_chunk = param_sets[(S_all_chunk*max_networks_per_S_all):]
        else:
            param_sets_S_all_chunk = param_sets[(S_all_chunk*max_networks_per_S_all):((S_all_chunk+1)*max_networks_per_S_all)]
        print(f"Simulating M={m}, S_all_{S_all_chunk}")
        num_sim_chunks = math.ceil(param_sets_S_all_chunk.shape[0]/max_curves_per_chunk_to_simulate)
        S_all = np.zeros((t,m+num_combos_with_replacement(m,r=2),param_sets_S_all_chunk.shape[0]))
        sim_futures = []
        for chunk in range(num_sim_chunks):
            if chunk==num_sim_chunks:
                param_sets_temp = param_sets_S_all_chunk[(chunk*max_curves_per_chunk_to_simulate):]
            else:
                param_sets_temp = param_sets_S_all_chunk[(chunk*max_curves_per_chunk_to_simulate):((chunk+1)*max_curves_per_chunk_to_simulate)]
            sim_futures.append(simulate_networks.remote(m = m, num_inputs = num_inputs, t = t, param_sets=param_sets_temp, input_lb = input_lb, \
                                input_ub = input_ub))
        for chunk,future in zip(range(num_sim_chunks),sim_futures):
            _, S_all_temp = ray.get(future)
            if chunk==num_sim_chunks:
                S_all[:,:,(chunk*max_curves_per_chunk_to_simulate):] = S_all_temp
            else:
                S_all[:,:,(chunk*max_curves_per_chunk_to_simulate):((chunk+1)*max_curves_per_chunk_to_simulate)] = S_all_temp
    
        del sim_futures
        np.save(str(pathlib.Path(specific_out_dir,f'S_all_{S_all_chunk}.npy')), S_all)
    
        ###### Process data
        S_all_processed = S_all.copy()
        # Reorder
        S_all_processed = S_all_processed[:,m:,:].reshape((t,-1),order='F').T # Exclude all monomers
        # Round low concentrations
        S_all_processed[np.where(S_all_processed<10**input_lb)] = round_low_concs_to
        # Calculate dynamic ranges, throw out curves with low dynamic ranges
        dynamic_ranges = np.apply_along_axis(get_dynamic_range, axis=1, arr=S_all_processed)
        S_all_processed_filtered = S_all_processed[np.where(dynamic_ranges>min_dynamic_range)[0],:]
        np.save(str(pathlib.Path(specific_out_dir,f'S_all_processed_filtered_{S_all_chunk}.npy')), S_all_processed_filtered)
        S_all_processed_filtered_files.append(str(pathlib.Path(specific_out_dir,f'S_all_processed_filtered_{S_all_chunk}.npy')))
        all_curves_per_Sall_chunk.append(S_all_processed.shape[0])
        if S_all_chunk==num_S_all_chunks:
            unfiltered_curves_per_Sall_chunk.append(np.sum(dimers_by_param_set[(S_all_chunk*max_networks_per_S_all):,:]))
        else:
            unfiltered_curves_per_Sall_chunk.append(np.sum(dimers_by_param_set[(S_all_chunk*max_networks_per_S_all):((S_all_chunk+1)*max_networks_per_S_all),:]))
        
        nonzero_curves_per_Sall_chunk.append(np.where(np.all(S_all_processed==round_low_concs_to,axis=1))[0].shape[0])
        filtered_curves_per_Sall_chunk.append(S_all_processed_filtered.shape[0])

        # Calculate map between original parameter sets and those kept after filtering
        # Allows us to trace curves in clusters back to original parameter sets
        univ_after_filter = np.where(dynamic_ranges>=min_dynamic_range)[0]
        num_univ_after_filter = np.where(dynamic_ranges>=min_dynamic_range)[0].shape[0]
        univ_filter_map = np.concatenate((np.expand_dims(univ_after_filter, axis=1),np.expand_dims(np.arange(num_univ_after_filter),axis=1)),axis=1)
        np.save(str(pathlib.Path(specific_out_dir,f'univ_filter_map_{S_all_chunk}.npy')), univ_filter_map)
    
    S_all_filenames_by_m.append(S_all_processed_filtered_files)
    all_curves_per_Sall_chunk_by_m.append(all_curves_per_Sall_chunk)
    unfiltered_curves_per_Sall_chunk_by_m.append(unfiltered_curves_per_Sall_chunk)
    nonzero_curves_per_Sall_chunk_by_m.append(nonzero_curves_per_Sall_chunk)
    filtered_curves_per_Sall_chunk_by_m.append(filtered_curves_per_Sall_chunk)

np.save(str(pathlib.Path(out_dir,'S_all_filenames_by_m.npy')),S_all_filenames_by_m)
np.save(str(pathlib.Path(out_dir,'all_curves_per_Sall_chunk_by_m.npy')),all_curves_per_Sall_chunk_by_m)
np.save(str(pathlib.Path(out_dir,'unfiltered_curves_per_Sall_chunk_by_m.npy')),unfiltered_curves_per_Sall_chunk_by_m)
np.save(str(pathlib.Path(out_dir,'nonzero_curves_per_Sall_chunk_by_m.npy')),nonzero_curves_per_Sall_chunk_by_m)
np.save(str(pathlib.Path(out_dir,'filtered_curves_per_Sall_chunk_by_m.npy')),filtered_curves_per_Sall_chunk_by_m)


## Perform 2D Screen

For the screen of two-input functions, we will just reuse a subset of parameter sets from the 1D screen

In [ ]:
# Define directory of 1D screen to load parameter sets from
param_set_load_dir = str(pathlib.Path('C:\\Users','jacob','Downloads'))

In [ ]:
# Define directory to save to
out_dir = str(pathlib.Path('C:\\Users','jacob','Downloads'))

if not os.path.isdir(out_dir):
    os.mkdir(out_dir)

In [ ]:
param_sets_per_m_approx = 250000 # Approximate number of parameter sets to simulate for each network size
K_log_range = [-5,7] # Lower and upper bounds for log10 of Kij values
A_log_range = [-3,3] # Lower and upper bounds for log10 of accessory monomer concentrations
input_lb = -3 # Lower bound for log10 of input concentrations
input_ub = 3 # Upper bound for log10 of input concentrations
t = 12 # Number of input concentrations to simulate
min_dynamic_range = 10 # Minimum dynamic range of the output responses (all other responses will be filtered out)
round_low_concs_to = 10**-3 # Concentrations below this value will be rounded to this value
max_curves_per_chunk_to_simulate=5000 # Maximum number of curves to simulate in a single "chunk"
max_networks_per_S_all=40000 # Maximum number of networks to save in a single S_all file

num_inputs = 2

In [ ]:
m_list = list(range(2,13)) # Network sizes to simulate

In [ ]:
S_all_filenames_by_m = []
all_curves_per_Sall_chunk_by_m = []
unfiltered_curves_per_Sall_chunk_by_m = []
nonzero_curves_per_Sall_chunk_by_m = []
filtered_curves_per_Sall_chunk_by_m = []

for m in m_list:
    specific_out_dir = str(pathlib.Path(out_dir,f'{m}M'))
    if not os.path.isdir(specific_out_dir):
        os.mkdir(specific_out_dir)
    rng = np.random.default_rng(seed=42)
    # Sample Param Sets
    print(f"Sampling parameter sets for m={m}")
    all_past_param_sets = np.load(str(pathlib.Path(param_set_load_dir,f'{m}M','K_A_param_sets.npy')),allow_pickle=True)
    num_possible_num_edges = num_combos(m,r=2) - (m-1) + 1
    num_sets_per_num_edges_thisset = math.ceil(param_sets_per_m_approx/num_possible_num_edges)
    num_sets_per_num_edges_fullset = math.ceil(all_past_param_sets.shape[0]/num_possible_num_edges)
    num_sets = int(num_sets_per_num_edges_thisset*num_possible_num_edges)
    param_sets = np.empty((num_sets,all_past_param_sets.shape[1]))
    for num_edges_i,num_edges in enumerate(range(m-1,num_combos(m,r=2)+1)):
        # For each connectivity level, sample a subset of parameter sets to use in the 2D screen
        param_sets[(num_edges_i*num_sets_per_num_edges_thisset):((num_edges_i+1)*num_sets_per_num_edges_thisset),:]=\
                    all_past_param_sets[rng.choice(np.arange((num_edges_i*num_sets_per_num_edges_fullset),((num_edges_i+1)*num_sets_per_num_edges_fullset)),\
                    size=(num_sets_per_num_edges_thisset,),replace=False)]
    # Remove M2 info
    param_sets = np.delete(param_sets,obj=num_combos_with_replacement(m,2),axis=1)
    dimers_by_param_set = np.apply_along_axis(dimers_from_param_set,\
                                                  axis=1,\
                                                  arr=param_sets,\
                                                  edges_to_count=list(range(num_combos_with_replacement(m,2))))
    np.save(str(pathlib.Path(specific_out_dir,f'K_log_range.npy')),np.array(K_log_range))
    np.save(str(pathlib.Path(specific_out_dir,f'A_log_range.npy')),np.array(A_log_range))
    np.save(str(pathlib.Path(specific_out_dir,f'K_A_param_sets.npy')), param_sets)
    np.save(str(pathlib.Path(specific_out_dir,f'dimers_by_param_set.npy')),dimers_by_param_set)
    # Chunk & simulate in parallel
    # There are two levels of chunking here:
    # 1. Each S_all matrix will only consider ~40,000 networks (for memory reasons)
    # 2. We will further chunk that up for simulation to allow parallelization
    num_S_all_chunks = math.ceil(param_sets.shape[0]/max_networks_per_S_all)
    S_all_processed_filtered_files = []
    all_curves_per_Sall_chunk = []
    unfiltered_curves_per_Sall_chunk = []
    nonzero_curves_per_Sall_chunk = []
    filtered_curves_per_Sall_chunk = []
    for S_all_chunk in range(num_S_all_chunks):
        if S_all_chunk==num_S_all_chunks:
            param_sets_S_all_chunk = param_sets[(S_all_chunk*max_networks_per_S_all):]
        else:
            param_sets_S_all_chunk = param_sets[(S_all_chunk*max_networks_per_S_all):((S_all_chunk+1)*max_networks_per_S_all)]
        print(f"Simulating M={m}, S_all_{S_all_chunk}")
        num_sim_chunks = math.ceil(param_sets_S_all_chunk.shape[0]/max_curves_per_chunk_to_simulate)
        S_all = np.zeros((t**2,m+num_combos_with_replacement(m,r=2),param_sets_S_all_chunk.shape[0]))
        sim_futures = []
        for chunk in range(num_sim_chunks):
            if chunk==num_sim_chunks:
                param_sets_temp = param_sets_S_all_chunk[(chunk*max_curves_per_chunk_to_simulate):]
            else:
                param_sets_temp = param_sets_S_all_chunk[(chunk*max_curves_per_chunk_to_simulate):((chunk+1)*max_curves_per_chunk_to_simulate)]
            sim_futures.append(simulate_networks.remote(m = m, num_inputs=num_inputs,t = t, param_sets=param_sets_temp, \
                                                                input_lb = input_lb, input_ub = input_ub))
        for chunk,future in zip(range(num_sim_chunks),sim_futures):
            _, S_all_temp = ray.get(future)
            if chunk==num_sim_chunks:
                S_all[:,:,(chunk*max_curves_per_chunk_to_simulate):] = S_all_temp
            else:
                S_all[:,:,(chunk*max_curves_per_chunk_to_simulate):((chunk+1)*max_curves_per_chunk_to_simulate)] = S_all_temp
    
        del sim_futures
        np.save(str(pathlib.Path(specific_out_dir,f'S_all_{S_all_chunk}.npy')), S_all)
    
        ###### Process data
        S_all_processed = S_all.copy()
        # Reorder
        S_all_processed = S_all_processed[:,m:,:].reshape(((t**num_inputs),-1),order='F').T # Exclude all monomers
        # Round low concentrations
        S_all_processed[np.where(S_all_processed<10**input_lb)] = round_low_concs_to
        # Calculate dynamic ranges, throw out curves with low dynamic ranges
        dynamic_ranges = np.apply_along_axis(get_dynamic_range, axis=1, arr=S_all_processed)
        S_all_processed_filtered = S_all_processed[np.where(dynamic_ranges>min_dynamic_range)[0],:]
        np.save(str(pathlib.Path(specific_out_dir,f'S_all_processed_filtered_{S_all_chunk}.npy')), S_all_processed_filtered)
        S_all_processed_filtered_files.append(str(pathlib.Path(specific_out_dir,f'S_all_processed_filtered_{S_all_chunk}.npy')))
        all_curves_per_Sall_chunk.append(S_all_processed.shape[0])
        if S_all_chunk==num_S_all_chunks:
            unfiltered_curves_per_Sall_chunk.append(np.sum(dimers_by_param_set[(S_all_chunk*max_networks_per_S_all):,:]))
        else:
            unfiltered_curves_per_Sall_chunk.append(np.sum(dimers_by_param_set[(S_all_chunk*max_networks_per_S_all):((S_all_chunk+1)*max_networks_per_S_all),:]))
        nonzero_curves_per_Sall_chunk.append(np.where(~np.all(S_all_processed==round_low_concs_to,axis=1))[0].shape[0])
        filtered_curves_per_Sall_chunk.append(S_all_processed_filtered.shape[0])

        # Calculate map between original parameter sets and those kept after filtering
        # Allows us to trace curves in clusters back to original parameter sets
        univ_after_filter = np.where(dynamic_ranges>=min_dynamic_range)[0]
        num_univ_after_filter = np.where(dynamic_ranges>=min_dynamic_range)[0].shape[0]
        univ_filter_map = np.concatenate((np.expand_dims(univ_after_filter, axis=1),np.expand_dims(np.arange(num_univ_after_filter),axis=1)),axis=1)
        np.save(str(pathlib.Path(specific_out_dir,f'univ_filter_map_{S_all_chunk}.npy')), univ_filter_map)
    S_all_filenames_by_m.append(S_all_processed_filtered_files)
    all_curves_per_Sall_chunk_by_m.append(all_curves_per_Sall_chunk)
    unfiltered_curves_per_Sall_chunk_by_m.append(unfiltered_curves_per_Sall_chunk)
    nonzero_curves_per_Sall_chunk_by_m.append(nonzero_curves_per_Sall_chunk)
    filtered_curves_per_Sall_chunk_by_m.append(filtered_curves_per_Sall_chunk)

np.save(str(pathlib.Path(out_dir,'S_all_filenames_by_m.npy')),S_all_filenames_by_m)
np.save(str(pathlib.Path(out_dir,'all_curves_per_Sall_chunk_by_m.npy')),all_curves_per_Sall_chunk_by_m)
np.save(str(pathlib.Path(out_dir,'unfiltered_curves_per_Sall_chunk_by_m.npy')),unfiltered_curves_per_Sall_chunk_by_m)
np.save(str(pathlib.Path(out_dir,'nonzero_curves_per_Sall_chunk_by_m.npy')),nonzero_curves_per_Sall_chunk_by_m)
np.save(str(pathlib.Path(out_dir,'filtered_curves_per_Sall_chunk_by_m.npy')),filtered_curves_per_Sall_chunk_by_m)